## 1. PP-msvsr Introduction
Video super-resolution originates from image super-resolution, which aims to recover high-resolution (HR) images from one or more low resolution (LR) images. The difference between them is that the video is composed of multiple frames, so the video super-resolution usually uses the information between frames to repair. PP-MSVSR is a multi-stage VSR deep architecture,  with local fusion module, auxiliary loss and refined align module to refine the enhanced result progressively. Specifically, in order to strengthen the fusion of features across frames in feature propagation, a local fusion module is designed in stage-1 to perform local feature fusion before feature propagation. Moreover, an auxiliary loss in stage-2 is introduced to make the features obtained by the propagation module reserve more correlated information connected to the HR space, and introduced a refined align module in stage-3 to make full use of the feature information of the previous stage. Extensive experiments substantiate that PP-MSVSR achieves a promising performance of Vid4 datasets, which PSNR metric can achieve 28.13 with only 1.45M parameters.

The PP-MSVSR model is officially produced by PaddlePaddle and is a video super-resolution model developed by PaddleGan. More information about PaddleGAN can be found here https://github.com/PaddlePaddle/PaddleGAN.


## 2. Model Effects and Application Scenarios
### 2.1 Video Super-Resolution Tasks:

#### 2.1.1 Datasets:

The commonly used video super-resolution dataset Vid4 is taken as an example.

#### 2.1.2 Model Effects:

PP-MSVSR在图片上的超分效果为：
The video super-resolution effect of PP-msvsr on the video is:

<div align='center'>
<img src='https://user-images.githubusercontent.com/48054808/144848981-00c6ad21-0702-4381-9544-becb227ed9f0.gif' width = "80%"  />
</div>


## 3. How to Use the Model

### 3.1 Model Inference:
* Download 


In [ ]:
%cd /home/aistudio/work
!git clone https://github.com/PaddlePaddle/PaddleGAN.git

* Installation

In [ ]:
# The script needs to be run in the PaddleGAN directory
%cd /home/aistudio/work/PaddleGAN/

# Install the required dependencies [already persisted, no need to install again].
!pip install -r requirements.txt

# The script needs to be run in the PaddleGAN directory
%cd /home/aistudio/work/PaddleGAN/

# Download PaddleGAN 
!python setup.py develop  

* Quick experience

Congratulations! Now that you've successfully installed PaddleGAN, let's get a quick feel at video super-resolution.

In [ ]:
# Predict a video on the GPU.
# Low resolution video download address: https://user-images.githubusercontent.com/79366697/200290225-7fdd364c-2fbe-48b6-a3bf-87349aedec98.mp4
%cd ~/work/PaddleGAN/applications/
!python tools/video-enhance.py --input demo/Peking_input360p_clip6_5s.mp4 \
                               --process_order PPMSVSR \
                               --output output_dir

A video with the predicted result is generated under the output_dir folder.


### 3.2 Model Training
* Clone the PaddleGAN repository (see 3.1 for details)

* Prepare the datasets

Here are 4 commonly used video super-resolution dataset, REDS, Vimeo90K, Vid4, UDM10. The REDS and Vimeo90K dataset include train dataset and test dataset, Vid4 and UDM10 are test dataset. Download and decompress the required dataset and place it under the ``PaddleGAN/data``.

REDS（[download](https://seungjunnah.github.io/Datasets/reds.html)）is a newly proposed high-quality (720p) video dataset in the NTIRE19 Competition. REDS consists of 240 training clips, 30 validation clips and 30 testing clips (each with 100 consecutive frames). Since the test ground truth is not available, we select four representative clips (they are '000', '011', '015', '020', with diverse scenes and motions) as our test set, denoted by REDS4. The remaining training and validation clips are re-grouped as our training dataset (a total of 266 clips).

The structure of the processed REDS is as follows:
```
  PaddleGAN
    ├── data
        ├── REDS
              ├── train_sharp
              |    └──X4
              ├── train_sharp_bicubic
              |    └──X4
              ├── REDS4_test_sharp
              |    └──X4
              └── REDS4_test_sharp_bicubic
                    └──X4
            ...
```

Vimeo90K ([download](http://toflow.csail.mit.edu/)) is designed by Tianfan Xue etc. for the following four video processing tasks: temporal frame interpolation, video denoising, video deblocking, and video super-resolution. Vimeo90K is a large-scale, high-quality video dataset. This dataset consists of 89,800 video clips downloaded from vimeo.com, which covers large variaty of scenes and actions.

The structure of the processed Vimeo90K is as follows:
```
  PaddleGAN
    ├── data
        ├── Vimeo90K
              ├── vimeo_septuplet
              |    |──sequences
              |    └──sep_trainlist.txt
              ├── vimeo_septuplet_BD_matlabLRx4
              |    └──sequences
              └── vimeo_super_resolution_test
                    |──low_resolution
                    |──target
                    └──sep_testlist.txt
            ...
```

Vid4 ([Data Download](https://paddlegan.bj.bcebos.com/datasets/Vid4.zip)) is a commonly used test dataset for VSR, which contains 4 video segments.
The structure of the processed Vid4 is as follows:
```
  PaddleGAN
    ├── data
        ├── Vid4
              ├── BDx4
              └── GT
            ...
```

UDM10 ([Data Download](https://paddlegan.bj.bcebos.com/datasets/udm10_paddle.tar)) is a commonly used test dataset for VSR, which contains 10 video segments.
The structure of the processed UDM10 is as follows:
```
  PaddleGAN
  ├── data
        ├── udm10
              ├── BDx4
              └── GT
            ...
```

Using the REDS dataset as an example, verify that the dataset is ready by using the following command.
 

In [ ]:
# Review the extract directory
#%cd /home/aistudio/work/PaddleGAN/
#!tree -d data/REDS

* Change yaml configurations files.

Change yaml configurations files``` configs/msvsr_reds.yaml```

```
total_iters: 150000  
output_dir: output_dir  
find_unused_parameters: True
checkpoints_dir: checkpoints
use_dataset: True
# tensor range for function tensor2img
min_max:
  (0., 1.)

model:
  name: MultiStageVSRModel
  fix_iter: 2500
  generator:
    name: MSVSR
    mid_channels: 32
    num_init_blocks: 2
    num_blocks: 3
    num_reconstruction_blocks: 2
    only_last: True
    use_tiny_spynet: True
    deform_groups: 4
    stage1_groups: 8
    auxiliary_loss: True
    use_refine_align: True
    aux_reconstruction_blocks: 1
    use_local_connnect: True
  pixel_criterion:
    name: CharbonnierLoss
    reduction: mean

dataset:
  train:
    name: RepeatDataset
    times: 1000
    num_workers: 6
    batch_size: 2  
    dataset:
      name: VSRREDSMultipleGTDataset
      lq_folder: data/REDS/train_sharp_bicubic/X4  
      gt_folder: data/REDS/train_sharp/X4  
      ann_file: data/REDS/meta_info_REDS_GT.txt  
      num_frames: 20  
      preprocess:
        - name: GetNeighboringFramesIdx
          interval_list: [1]
        - name: ReadImageSequence
          key: lq
        - name: ReadImageSequence
          key: gt
        - name: Transforms
          input_keys: [lq, gt]
          pipeline:
            - name: SRPairedRandomCrop
              gt_patch_size: 256
              scale: 4
              keys: [image, image]
            - name: PairedRandomHorizontalFlip
              keys: [image, image]
            - name: PairedRandomVerticalFlip
              keys: [image, image]
            - name: PairedRandomTransposeHW
              keys: [image, image]
            - name: TransposeSequence
              keys: [image, image]
            - name: NormalizeSequence
              mean: [0., 0., 0.]
              std: [255., 255., 255.]
              keys: [image, image]

  test:
    name: VSRREDSMultipleGTDataset
    lq_folder: data/REDS/REDS4_test_sharp_bicubic/X4  
    gt_folder: data/REDS/REDS4_test_sharp/X4  
    ann_file: data/REDS/meta_info_REDS_GT.txt 
    num_frames: 100  
    test_mode: True
    preprocess:
        - name: GetNeighboringFramesIdx
          interval_list: [1]
        - name: ReadImageSequence
          key: lq
        - name: ReadImageSequence
          key: gt
        - name: Transforms
          input_keys: [lq, gt]
          pipeline:
            - name: TransposeSequence
              keys: [image, image]
            - name: NormalizeSequence
              mean: [0., 0., 0.]
              std: [255., 255., 255.]
              keys: [image, image]

lr_scheduler:
  name: CosineAnnealingRestartLR
  learning_rate: !!float 2e-4  
  periods: [150000]
  restart_weights: [1]
  eta_min: !!float 1e-7

optimizer:
  name: Adam
  # add parameters of net_name to optim
  # name should in self.nets
  net_names:
    - generator
  beta1: 0.9
  beta2: 0.99

validate:
  interval: 5000 
  save_img: false 

  metrics:
    psnr: # metric name, can be arbitrary
      name: PSNR  
      crop_border: 0
      test_y_channel: false
    ssim:
      name: SSIM  
      crop_border: 0
      test_y_channel: false

log_config:
  interval: 10  
  visiual_interval: 5000 

snapshot_config:
  interval: 5000 

export_model:
  - {name: 'generator', inputs_num: 1}

```


* Train the model.

In [ ]:
%cd ~/work/PaddleGAN/
%export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7,8
# Beginning training
!ppython -m paddle.distributed.launch tools/main.py --config-file configs/msvsr_reds.yaml

* Model evaluation

We provide ```configs/msvsr_reds.yaml```for evaluating the effect of REDS dataset, to evaluate the effect of REDS dataset, you must first download the REDS dataset from the REDS dataset download page, and extract it to ```PaddleGAN/data/REDS```. Using the following command can evaluate.

In [ ]:
%cd ~/work/PaddleGAN/
%env CUDA_VISIBLE_DEVICES=0
#Download address for model: https://paddlegan.bj.bcebos.com/models/PP-MSVSR_reds_x4.pdparams
!python tools/main.py --config-file configs/msvsr_reds.yaml --evaluate-only --load ${PATH_OF_WEIGHT}

## 4. Model Principles


* Different from the Single Image Super-Resolution(SISR) task, the key for Video Super-Resolution(VSR) task is to make full use of complementary information across frames to reconstruct the high-resolution sequence. Since images from different frames with diverse motion and scene, accurately aligning multiple frames and effectively fusing different frames has always been the key research work of VSR tasks. To utilizes rich complementary information of neighboring frames, PP-MSVSR propose a multi-stage VSR deep architecture, with local fusion module, auxiliary loss and refined align module to refine the enhanced result progressively. 
* PP-MSVSR propose a multi-stage network that combines the idea of sliding-window framework and recurrent framework, with local fusion module, auxiliary loss and refined align module to refine the enhanced result progressively. 
<div align="center">
<img src="https://user-images.githubusercontent.com/79366697/200302378-cd0a5f1b-5d89-44a5-aae3-8f4c587a3575.png" width = "80%"  />
</div>

* Inspired by the idea of sliding-window VSR, PP-MSVSR designed a local fusion module in stage-1, denoted as LFM, to perform local feature fusion before feature propagation, which can strengthen cross-frame feature fusion in feature propagation. Specifically, the purpose of the LFM is to let the features of the current frame fuse the information of its neighboring frames first, and then send the fused features to the propagation module.
<div align="center">
<img src="https://user-images.githubusercontent.com/79366697/200302612-6ac31db1-ac32-4b14-af3e-479f66746ac9.png" width = "80%"  />
</div>

* Inspired by the power of recurrent vsr network, PP-MSVSR use a same structure as basicvsr++ to fusion the information from different video frame and local merged feature and then propagates the underlying information between each video frame at the stage-2. In addition, PP-MSVSR add a auxiliary loss to make feature more closed to HR space. To be specific, the auxiliary loss is used after upsampling the feature, which propagation in stage-2
* Different from SISR, in order to better integrate the in- formation of adjacent frames, VSR usually aligns adjacent frames with the current frame. In some large motion video restoration tasks, the role of alignment is particularly ob- vious. In the process of using a two-way recurrent neural network, there are often multiple identical alignment opera- tions. In order to make full use of the results of the previous alignment operations, PP-MSVSR propose a Refined Align Module, denoted as RAM, that can utilize the previously aligned parameters and achieve a better alignment result.
<div align="center">
<img src="https://user-images.githubusercontent.com/79366697/200302619-87b10406-580b-4a65-a6bd-bbb1525005b1.png" width = "80%"  />
</div>


## 5. Related papers and citations
```
@article{jiang2021PP-MSVSR,
  author = {Jiang, Lielin and Wang, Na and Dang, Qingqing and Liu, Rui and Lai, Baohua},
  title = {PP-MSVSR: Multi-Stage Video Super-Resolution},
  booktitle = {arXiv preprint arXiv:2112.02828},
  year = {2021}
  }
```
